In [3]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]━━ 1/2 [openpyxl]


In [4]:
import pandas as pd
from pathlib import Path

product_anjing_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Product List Anjing.xlsx")

product_anjing_list = pd.read_excel(product_anjing_path)
product_anjing_list.head()

,product_name
0,Blackfoot keranjang belanja lipat 4 roda shopp...
1,Blackfoot keranjang belanja lipat 4 roda shopp...
2,BLACKFOOT shopping cart 2 roda keranjang belan...
3,mesin blower angin RICHU & KODENKI high speed ...
4,Mesin blower angin semprotan bulu anjing tekan...


In [1]:
forbidden_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Keyword: Forbidden Word.csv")
forbidden_list = pd.read_csv(forbidden_path)
forbidden_list.head()

NameError: name 'Path' is not defined

In [14]:
!pip install openai==0.28.1

  Attempting uninstall: openai━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━ 6/8 [aiohttp]
    Found existing installation: openai 1.98.0━╺━━━━━━━━━ 6/8 [aiohttp]
    Uninstalling openai-1.98.0:━━━━━━━━━━━━━━━━╺━━━━━━━━━ 6/8 [aiohttp]
      Successfully uninstalled openai-1.98.0━━━╺━━━━━━━━━ 6/8 [aiohttp]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openai]━━━━ 7/8 [openai]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
context-violation-detector 0.1.0 requires openai<2.0.0,>=1.98.0, but you have openai 0.28.1 which is incompatible.


In [9]:
import openai
import pandas as pd
#from project.context_violation_detection.config.settings import GROQ_API_KEY
from pathlib import Path
import time
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("GROQ_API_KEY")
openai.api_base = "https://api.groq.com/openai/v1"

# Fungsi pengecekan
def check_violation(product_name: str, forbidden_words: list[str]) -> bool:
    prompt = f"""
You are a content validator. Your job is to detect if a product name violates content policy by containing forbidden words in a harmful or inappropriate context.

Forbidden words: {", ".join(forbidden_words)}

Example (does NOT violate): 'kandang anjing'
Example (does NOT violate): 'alat makan anjing anti tumpah'
Example (does NOT violate): 'sabun penghilang bau kotoran anjing'

Example (DOES violate): 'lu kayak anjing'
Example (DOES violate): 'bener bener anjing lu'
Example (DOES violate): 'jual bokep murah'

Product name: '{product_name}'

Does it violate the content policy? Answer with "YES" or "NO" only.
"""

    response = openai.ChatCompletion.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=3,
    )

    return "YES" in response.choices[0].message["content"].upper()

# Load data
csv_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Product List 1000.csv")
product_list = pd.read_csv(csv_path)

forbidden_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Keyword: Forbidden Word.csv")
forbidden_df = pd.read_csv(forbidden_path)

# Ubah ke list dan lowercase
forbidden_words = [str(word).lower() for word in forbidden_df.iloc[:, 0].dropna().unique()]

# Cek pelanggaran
results = []
for name in product_list['product_name']:
    try:
        is_violation = check_violation(str(name), forbidden_words)
        results.append({"product_name": name, "violation": is_violation})
        time.sleep(1.2)  # Hindari rate limit (Groq lebih cepat, tapi amankan saja)
    except Exception as e:
        results.append({"product_name": name, "violation": None, "error": str(e)})

# Convert ke DataFrame
violation_df = pd.DataFrame(results)

# Simpan hasil jika perlu
violation_df.to_csv("violating_products.csv", index=False)
violation_df.head()

,product_name,violation,error
0,Sharp LED Digital 2K 2T-C24HD1500i / 2T C24HD ...,None,"\n\nYou tried to access openai.ChatCompletion,..."
1,RAK GALON KRISBOW 4 TINGKAT,None,"\n\nYou tried to access openai.ChatCompletion,..."
2,KRISHOME TROLI RAK 4 TINGKAT / TROLI DORONG SE...,None,"\n\nYou tried to access openai.ChatCompletion,..."
3,iPhone 13 128GB,None,"\n\nYou tried to access openai.ChatCompletion,..."
4,ASOBU Bestie Bottle - Botol Minum / Tumbler / ...,None,"\n\nYou tried to access openai.ChatCompletion,..."


In [19]:
!pip install python-dotenv openai

In [23]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

def check_violation(prompt: str, model: str = "llama3-70b-8192") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Only answer YES or NO."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
        max_tokens=3,
    )
    return response.choices[0].message.content.strip().upper()

# Cek pelanggaran
results = []
for name in product_list['product_name']:
    try:
        is_violation = check_violation(str(name), forbidden_words)
        results.append({"product_name": name, "violation": is_violation})
        time.sleep(1.2)  # Hindari rate limit (Groq lebih cepat, tapi amankan saja)
    except Exception as e:
        results.append({"product_name": name, "violation": None, "error": str(e)})

# Convert ke DataFrame
violation_df = pd.DataFrame(results)

# Simpan hasil jika perlu
violation_df.to_csv("violating_products.csv", index=False)
violation_df.head()

,product_name,violation,error
0,Sharp LED Digital 2K 2T-C24HD1500i / 2T C24HD ...,None,No module named 'openai.resources'
1,RAK GALON KRISBOW 4 TINGKAT,None,No module named 'openai.resources'
2,KRISHOME TROLI RAK 4 TINGKAT / TROLI DORONG SE...,None,No module named 'openai.resources'
3,iPhone 13 128GB,None,No module named 'openai.resources'
4,ASOBU Bestie Bottle - Botol Minum / Tumbler / ...,None,No module named 'openai.resources'


In [2]:
import openai
print(openai.__file__)
print(openai.__version__)

/Users/vetybhakti/Documents/Renos/context-violation-detection/.venv/lib/python3.12/site-packages/openai/__init__.py
0.28.1


In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os, time

load_dotenv()
client = OpenAI(
  api_key=os.getenv("GROQ_API_KEY"),
  base_url="https://api.groq.com/openai/v1"
)

def check_violation(product_name: str, forbidden_words: list[str], model="mixtral-8x7b-32768") -> str:
    prompt = f"""
Forbidden words: {', '.join(forbidden_words)}
Example (does NOT violate): ...
Example (DOES violate): ...

Product name: '{product_name}'
Answer YES or NO."""
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1, max_tokens=3
    )
    return resp.choices[0].message.content.strip().upper()

# Cek pelanggaran
results = []
for name in product_list['product_name']:
    try:
        is_violation = check_violation(str(name), forbidden_words)
        results.append({"product_name": name, "violation": is_violation})
        time.sleep(1.2)  # Hindari rate limit (Groq lebih cepat, tapi amankan saja)
    except Exception as e:
        results.append({"product_name": name, "violation": None, "error": str(e)})

# Convert ke DataFrame
violation_df = pd.DataFrame(results)

# Simpan hasil jika perlu
violation_df.to_csv("violating_products.csv", index=False)
violation_df.head()

NameError: name 'product_list' is not defined

In [11]:
import pandas as pd
import time
from pathlib import Path
from openai import OpenAI
from decouple import config

# Inisialisasi client Groq
client = OpenAI(
    api_key=config("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# Function untuk cek pelanggaran
def check_violation(product_name, forbidden_words):
    prompt = (
        f"Given the product name: \"{product_name}\", "
        f"check if it contains any of these forbidden words: {', '.join(forbidden_words)}. "
        f"Return 'yes' if it contains, 'no' otherwise."
    )
    
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=3,
        temperature=0.1,
    )

    reply = response.choices[0].message.content.lower()
    return "yes" in reply

# Path ke file CSV
product_anjing_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Product List Anjing.xlsx")

product_anjing_list = pd.read_excel(product_anjing_path)

# Daftar kata terlarang (contoh)
forbidden_words = [
    "sex", "obat kuat", "anjing"
]

# Batasi hanya 5 kata terlarang saat eksplorasi
forbidden_words = forbidden_words[:5]

# Proses cek pelanggaran
results = []
for name in product_anjing_list['product_name']:
    try:
        is_violation = check_violation(str(name), forbidden_words)
        print(f"Checking: {name[:50]}... -> {is_violation}")
        results.append({"product_name": name, "violation": is_violation})
        time.sleep(1.2)  # Hindari rate limit
    except Exception as e:
        print(f"Error for '{name[:50]}': {e}")
        results.append({"product_name": name, "violation": None, "error": str(e)})

# Simpan ke CSV hasilnya
violation_df = pd.DataFrame(results)
violation_df.to_csv("violating_products.csv", index=False)

# Tampilkan hasil awal
print(violation_df.head())

Checking: Blackfoot keranjang belanja lipat 4 roda shopping ... -> False
Checking: Blackfoot keranjang belanja lipat 4 roda shopping ... -> False
Checking: BLACKFOOT shopping cart 2 roda keranjang belanja l... -> False
Checking: mesin blower angin RICHU & KODENKI high speed hand... -> False
Checking: Mesin blower angin semprotan bulu anjing tekanan t... -> False
Checking: Tempat makan anjing kucing mangkuk stainless... -> False
Checking: Tempat tidur anjing kucing hewan peliharaan kayu j... -> False
Checking: Rumah kucing / Rumah anjing hewan peliharaan kayu ... -> False
Checking: Trendy Pet Bed (S) Anti Slip Busa Rebonded (Tempat... -> False
Checking: Trendy Cover / Sarung Pet Bed (M) (Tempat tidur an... -> False
Checking: Trendy Cover / Sarung Pet Bed (L) (Tempat tidur an... -> False
Checking: Trendy Cover / Sarung Pet Bed (S) (Tempat tidur an... -> False
Checking: Anjing lo... -> False
Checking: Bener bener kayak anjing lu... -> False
                                        product_

In [12]:
import pandas as pd
import time
from pathlib import Path
from openai import OpenAI
from decouple import config

# Inisialisasi Groq client
client = OpenAI(
    api_key=config("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# Fungsi untuk deteksi pelanggaran berbasis konteks
def check_violation(product_name, forbidden_words):
    prompt = (
        f"Product name: \"{product_name}\"\n"
        f"Forbidden words (reference only): {', '.join(forbidden_words)}\n\n"
        f"Determine whether this product name contains hate speech, vulgar, or offensive language "
        f"even if the words are used as part of a longer sentence or in an inappropriate context.\n"
        f"Reply only with 'yes' or 'no'."
    )

    try:
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "You are a content moderation assistant that detects inappropriate language in product names."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=5,
            temperature=0.1,
        )

        reply = response.choices[0].message.content.strip().lower()
        return reply.startswith("yes")

    except Exception as e:
        print(f"API error for \"{product_name[:40]}...\": {e}")
        return None

# Path ke file produk
product_anjing_path = Path("/Users/vetybhakti/Documents/Renos/context-violation-detection/project/context_violation_detection/sample_data/Product List Anjing.xlsx")
product_anjing_list = pd.read_excel(product_anjing_path)

# Daftar kata terlarang (bisa diperluas)
forbidden_words = ["sex", "obat kuat", "anjing"][:5]

# Proses pelanggaran
results = []
for name in product_anjing_list['product_name']:
    name_str = str(name)
    is_violation = check_violation(name_str, forbidden_words)
    print(f"Checking: {name_str[:50]:<50} -> {is_violation}")
    results.append({"product_name": name_str, "violation": is_violation})
    time.sleep(1.2)  # Hindari rate limit

# Simpan hasil ke CSV
violation_df = pd.DataFrame(results)
violation_df.to_csv("violating_products.csv", index=False)

# Print 5 hasil pertama
print(violation_df.head())

Checking: Blackfoot keranjang belanja lipat 4 roda shopping  -> False
Checking: Blackfoot keranjang belanja lipat 4 roda shopping  -> False
Checking: BLACKFOOT shopping cart 2 roda keranjang belanja l -> False
Checking: mesin blower angin RICHU & KODENKI high speed hand -> False
Checking: Mesin blower angin semprotan bulu anjing tekanan t -> False
Checking: Tempat makan anjing kucing mangkuk stainless       -> True
Checking: Tempat tidur anjing kucing hewan peliharaan kayu j -> False
Checking: Rumah kucing / Rumah anjing hewan peliharaan kayu  -> False
Checking: Trendy Pet Bed (S) Anti Slip Busa Rebonded (Tempat -> False
Checking: Trendy Cover / Sarung Pet Bed (M) (Tempat tidur an -> False
Checking: Trendy Cover / Sarung Pet Bed (L) (Tempat tidur an -> False
Checking: Trendy Cover / Sarung Pet Bed (S) (Tempat tidur an -> False
Checking: Anjing lo                                          -> True
Checking: Bener bener kayak anjing lu                        -> True
                       